In [203]:
import torch, math
import torch.nn as nn
import torch.nn.functional as F

In [204]:
torch.manual_seed(0)
torch.random.manual_seed(0)
x = torch.randn(1, 1, 32, 32)

# 1

In [209]:
import torch, math
import torch.nn as nn
import torch.nn.functional as F


def second_order_polynomial(x_q, s_x, a, b, c):
    """a(x+b)^2 + c"""
    """Kim, Sehoon, et al. "I-bert: Integer-only bert quantization." International conference on machine learning. PMLR, 2021."""
    with torch.no_grad():
        # [ Can be pre-computed ]
        q_b = torch.floor(b / s_x)  # PRE-COMPUTED >> INT
        q_c = torch.floor(c / (a * s_x**2))  # PRE-COMPUTED >> INT
        s_out = a * s_x**2  # PRE-COMPUTED-FP

    q_out = (x_q + q_b).pow(2) + q_c

    return q_out, s_out


def int_ERF(x_q, s_x):
    with torch.no_grad():
        a, b, c = -0.2888, -1.769, 1
        a, b, c = torch.tensor(a), torch.tensor(b), torch.tensor(c)

    """ONLY INT OPERATION"""
    _q = torch.min(x_q.abs(), -b / s_x)
    print(-b / s_x)

    q_L, s_L = second_order_polynomial(_q, s_x, a, b, c)

    q_out = torch.sign(x_q) * q_L

    return q_out, s_L


def int_GELU(x_q, s_x):
    with torch.device(x_q.device):
        # input : (INT, PRE-COMPUTED-FP)
        q_erf, s_erf = int_ERF(x_q, s_x / math.sqrt(2))
        # output : (INT, PRE-COMPUTED-FP)

        q_1 = torch.floor(1 / s_erf)  # floor(1 / PRE-COMPUTED-FP) >> INT

        q_out = x_q * (q_erf + q_1)  # INT
        s_out = s_x * s_erf / 2  # PRE-COMPUTED-FP

        return q_out, s_out

(-b/s) 얘의 반올림이 적용되냐 안되냐 차이였음. 적용 안 하는게 맞고, 0.04%p 높음

# 2

In [210]:
def i_ERF( q, S):
    with torch.no_grad():
        a, b, c = -0.2888, -1.769, 1
        a, b, c = torch.tensor(a), torch.tensor(b), torch.tensor(c)
        qb = torch.floor(b / S)  # PRE-COMPUTED >> INT
        qc = torch.floor(c / (a * S**2))  # PRE-COMPUTED >> INT

    """ONLY INT OPERATION"""
    _q = q.sign() * ((torch.min(q.abs(), -b/S) + qb).pow(2) + qc)
    print(-b/S)

    _S = a * S**2  # PRE-COMPUTED >> FP

    return _q, _S

def i_GELU( q, S):
    with torch.device(q.device):

        # input : (INT, PRE-COMPUTED-FP)
        q_erf, S_erf = i_ERF(q, S / math.sqrt(2))
        # output : (INT, PRE-COMPUTED-FP)

        q1 = torch.floor(1 / S_erf)  # floor(1 / PRE-COMPUTED-FP) >> INT

        _q = q * (q_erf + q1)  # INT
        _S = S * S_erf / 2  # PRE-COMPUTED-FP

        return _q, _S

In [211]:
print(x.min(), x.max())

tensor(-3.1537) tensor(4.1015)


In [212]:
import matplotlib.pyplot as plt


x1, s1 = int_GELU(x, 100)

x2 , s2= i_GELU(x, 100)

print(x1, s1)
print(x2, s2)

print(x1.shape, x2.shape)

torch.norm(x1.float() - x2.float())

tensor(0.0250)
tensor(0.0250)
tensor([[[[ 1.0702,  1.0954,  0.2382,  ..., -1.6647, -0.9931,  0.8020],
          [ 0.5833, -0.0332,  0.4683,  ...,  1.1732, -1.9096,  0.5243],
          [ 0.5411, -0.9654, -1.1657,  ...,  0.9092, -0.0352, -0.7452],
          ...,
          [-1.0669, -1.3503, -2.1849,  ..., -0.8565,  0.5429,  0.1136],
          [ 0.4062, -0.8546,  1.3423,  ...,  0.1325,  0.3495,  0.4348],
          [ 1.2306, -0.7358,  1.8154,  ..., -0.5640, -1.1361,  1.6260]]]]) tensor(-72200.)
tensor([[[[ 1.0702,  1.0954,  0.2382,  ..., -1.6647, -0.9931,  0.8020],
          [ 0.5833, -0.0332,  0.4683,  ...,  1.1732, -1.9096,  0.5243],
          [ 0.5411, -0.9654, -1.1657,  ...,  0.9092, -0.0352, -0.7452],
          ...,
          [-1.0669, -1.3503, -2.1849,  ..., -0.8565,  0.5429,  0.1136],
          [ 0.4062, -0.8546,  1.3423,  ...,  0.1325,  0.3495,  0.4348],
          [ 1.2306, -0.7358,  1.8154,  ..., -0.5640, -1.1361,  1.6260]]]]) tensor(-72200.)
torch.Size([1, 1, 32, 32]) torch.Size(

tensor(0.)